# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json 


# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = pd.read_csv("../output_data/cities.csv")

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [13]:
output_data_file.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


## Generate Cities List

In [14]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)


626

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [9]:
cities[:8]

['airai',
 'mwinilunga',
 'klaksvik',
 'takoradi',
 'albany',
 'yicheng',
 'oksfjord',
 'punta arenas']

In [62]:
#create empty lists
city_name = []
cloudiness = []
country = []
date = []
humidity = []
lat = []
lng = []
max_temp = []
wind_speed = []

#weather api 
url = 'http://api.openweathermap.org/data/2.5/weather'

#print statement
print('Beginning Data Retrieval')
print('-----------------------------')

record = 0
set_count = 1
for city in cities:
    #create parameters for api call 
    params={
        "q":city,
        "units":"imperial",
        "appid": weather_api_key
    }
    try:
        response = requests.get(url, params)
        #see if API call successful
        #print(response.url)
        results = response.json()
        #reformat to see data better
        #print(json.dumps(results, indent=4, sort_keys=True))
        cloudiness.append(results["clouds"]["all"])
        date.append(results["dt"])
        humidity.append(results["main"]["humidity"])
        lat.append(results["coord"]["lat"])
        lng.append(results["coord"]["lon"])
        max_temp.append(results["main"]["temp_max"])
        wind_speed.append(results["wind"]["speed"])
        #increase counter by one, include a set count 
        if record >=50:
            record = 0,
            set_count +=1
        print(f"Processing Record{record}|{city}")
    
    except Exception:
        print("City not found. Skipping...")

print("------------------------------")
print("Data Retrieval Complete")
print("------------------------------")

Beginning Data Retrieval
-----------------------------
Processing Record0|saint anthony
Processing Record0|luderitz
Processing Record0|kodiak
Processing Record0|avarua
Processing Record0|bengkulu
Processing Record0|nisia floresta
Processing Record0|hermanus
Processing Record0|lasa
Processing Record0|jamestown
Processing Record0|turayf
Processing Record0|yellowknife
Processing Record0|kapaa
Processing Record0|independence
Processing Record0|albany
Processing Record0|okitipupa
Processing Record0|denpasar
Processing Record0|nikolskoye
Processing Record0|atuona
Processing Record0|busselton
Processing Record0|opuwo
Processing Record0|bredasdorp
Processing Record0|yar-sale
Processing Record0|tilichiki
Processing Record0|hinton
Processing Record0|mataura
Processing Record0|barrow
Processing Record0|naze
Processing Record0|rosario
Processing Record0|rikitea
Processing Record0|orapa
Processing Record0|mana
Processing Record0|saint-philippe
Processing Record0|puerto ayora
Processing Record0|espe

Processing Record0|twentynine palms
Processing Record0|nanakuli
Processing Record0|ranong
Processing Record0|nhulunbuy
Processing Record0|aitape
Processing Record0|kortkeros
Processing Record0|srandakan
Processing Record0|kaeo
Processing Record0|vanavara
Processing Record0|nizip
Processing Record0|mogapinyana
Processing Record0|taltal
Processing Record0|coahuayana
Processing Record0|dingle
Processing Record0|ippy
Processing Record0|ostersund
Processing Record0|dubbo
Processing Record0|victoria
Processing Record0|ust-nera
Processing Record0|cidreira
Processing Record0|rocha
Processing Record0|jackson
Processing Record0|karratha
Processing Record0|fort nelson
Processing Record0|tigil
Processing Record0|inhambane
Processing Record0|bardiyah
Processing Record0|ambilobe
Processing Record0|xining
Processing Record0|mackay
Processing Record0|the valley
Processing Record0|udachnyy
Processing Record0|toowoomba
Processing Record0|hambantota
Processing Record0|pak phanang
Processing Record0|flin 

Processing Record0|kameda
Processing Record0|san carlos de bariloche
Processing Record0|shache
Processing Record0|vanimo
City not found. Skipping...
City not found. Skipping...
Processing Record0|skelleftea
Processing Record0|obo
Processing Record0|bougouni
Processing Record0|half moon bay
Processing Record0|margate
Processing Record0|praia da vitoria
Processing Record0|yangshan
Processing Record0|darasti-ilfov
Processing Record0|mount isa
Processing Record0|abadiania
Processing Record0|guadalupe y calvo
Processing Record0|sangar
Processing Record0|grajau
City not found. Skipping...
Processing Record0|quatre cocos
Processing Record0|green river
Processing Record0|sorong
Processing Record0|acajutla
Processing Record0|colon
Processing Record0|chapais
Processing Record0|helong
Processing Record0|ponta delgada
Processing Record0|bedford
Processing Record0|berdigestyakh
Processing Record0|port blair
Processing Record0|belaya gora
Processing Record0|jardim
Processing Record0|iracoubo
Process

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

#### Latitude vs. Humidity Plot

#### Latitude vs. Cloudiness Plot

#### Latitude vs. Wind Speed Plot

## Linear Regression

In [6]:
# OPTIONAL: Create a function to create Linear Regression plots


In [7]:
# Create Northern and Southern Hemisphere DataFrames


####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression